In [1]:
import os
CURDIR = os.getcwd().split("\\")
if "notebooks" in CURDIR:
    n = len(CURDIR) - CURDIR.index("notebooks")
    os.chdir(os.getcwd() + ("\\.." * n))

In [2]:
import seaborn as sns
import polars as pl
diamonds = pl.DataFrame(sns.load_dataset("diamonds"))

In [6]:
from prep.fct_helper import fct_levels, fct_ordered
# diamonds.cut = fct_ordered(diamonds.cut,ordered_levels=["Fair","Good","Very Good","Premium","Ideal"])
diamonds.with_columns(
    fct_ordered(pl.col("cut"),ordered_levels=["Fair","Good","Very Good","Premium","Ideal"])
).sort("cut")

carat,cut,color,clarity,depth,table,price,x,y,z
f64,cat,cat,cat,f64,f64,i64,f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.23,"""Ideal""","""J""","""VS1""",62.8,56.0,340,3.93,3.9,2.46
0.31,"""Ideal""","""J""","""SI2""",62.2,54.0,344,4.35,4.37,2.71
0.3,"""Ideal""","""I""","""SI2""",62.0,54.0,348,4.31,4.34,2.68
0.33,"""Ideal""","""I""","""SI2""",61.8,55.0,403,4.49,4.51,2.78
0.33,"""Ideal""","""I""","""SI2""",61.2,56.0,403,4.49,4.5,2.75
0.33,"""Ideal""","""J""","""SI1""",61.1,56.0,403,4.49,4.55,2.76
0.23,"""Ideal""","""G""","""VS1""",61.9,54.0,404,3.93,3.95,2.44
0.32,"""Ideal""","""I""","""SI1""",60.9,55.0,404,4.45,4.48,2.72


In [21]:
import polars as pl
pl_diamonds = diamonds
import pandas as pd
from typing import Union

def fct_ordered(series:Union[pd.Series,pl.Series], ordered_levels:list):
    """
    Create ordered factor from string or normal cat series
    Example: diamonds.cut = fct_ordered(diamonds.cut,ordered_levels=["Fair","Good","Very Good","Premium","Ideal"])
    """
    
    if isinstance(series, pd.Series):
        cat_type = CategoricalDtype(ordered_levels, ordered = True)
        return series.astype(cat_type)
    else:
        with pl.StringCache():
            print(ordered_levels)
            pl.Series(ordered_levels).cast(pl.Categorical)
            print( pl.Series(ordered_levels).cast(pl.Categorical))
            return series.cast(pl.Categorical).cat.set_ordering('physical')
        

pl_diamonds = pl_diamonds.with_columns(
    fct_ordered(pl.col("cut"),["Fair","Good","Very Good","Premium","Ideal"])
).sort("cut")

pl_diamonds.select(
    "cut",

    pl.col("cut").to_physical().alias("physical")
)


['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
shape: (5,)
Series: '' [cat]
[
	"Fair"
	"Good"
	"Very Good"
	"Premium"
	"Ideal"
]


cut,physical
cat,u32
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0
"""Ideal""",0


['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']

In [31]:
prep_freq_table(pl_diamonds, group_vars="cut").sort("cut")

cut,count,proportion
cat,u32,f64
"""Fair""",1610,0.029848
"""Good""",4906,0.090953
"""Very Good""",12082,0.22399
"""Premium""",13791,0.255673
"""Ideal""",21551,0.399537


cut
cat
"""Ideal"""
"""Premium"""
"""Good"""
"""Premium"""
"""Good"""
"""Very Good"""
"""Very Good"""
"""Very Good"""
"""Fair"""


In [15]:
from prep.prep_freq_table import prep_freq_table
import polars as pl
prep_freq_table(pl.DataFrame(diamonds).with_columns(pl.col("cut").cat.set_ordering('physical')), group_vars="cut")

cut,count,proportion
cat,u32,f64
"""Ideal""",21551,0.399537
"""Good""",4906,0.090953
"""Fair""",1610,0.029848
"""Very Good""",12082,0.22399
"""Premium""",13791,0.255673


In [22]:
pl.DataFrame(diamonds).select("cut").to_series().cat